## MATLAB Arm Visualization Control
While ROS has many great packages for robot visualiztion and simulation, we cant use them since we are running ROS in a jupyter notebook which does not have a desktop gui. To get around this and still give everyone a chance to iteract with a simulated robot using ROS, I developed a MATLAB program which launches a virtaul UR3e arm which can be controlled via ROS. It can be run in desktop MATLAB or MATLAB online. In the MATLAB program you will need to change the MQTT client name and pub/sub topics based on your group number.

### How it works
I created a ROS2 package which launches N versions of a node configured as an action server. It uses parameters to dynamically set MQTT client names, topics, and FollowJointTrajectory and gripper/control namespaces so each node can be linked to a single arm visualization. MQTT is used to connect to the simulation which is outside the ROS network (this is a little bit of a weird roundabout setup but not unheard of to use microcontrollers in ROS systems since they cant run the full thing). Also it means the same FollowJointTrajectory action client which is used to control the arm can be used to control the simulation just by changing the namespace of the action.

To get the MATLAB arm simulator function or see the inner workings of the package you can check out the [github repo](https://github.com/e-sarvey/myurviz).


In [ ]:
! ros2 node list # check the nodes availible

In [ ]:
! ros2 action list # check the actions availible

### Sending an action goal to the virtual arm with the terminal
To send a goal to the simulated arm, edit the CLI command below for your action server (eg. /simulated_arm_server_N/). You can also change the positions to alter the joint angles. Use the visualization interface to find some angles and build a trajectory to execute!

In [ ]:
# to move to a single pose [0, -3.1416, 1.5708, -1.5708, 1.0647, 0] over 5 seconds
! ros2 action send_goal /simulated_arm_server_N/follow_joint_trajectory \
  control_msgs/action/FollowJointTrajectory \
  "{trajectory: {joint_names: [\"shoulder_pan_joint\", \"shoulder_lift_joint\", \"elbow_joint\", \
  \"wrist_1_joint\", \"wrist_2_joint\", \"wrist_3_joint\"], \
  points: [{positions: [0, -3.1416, 1.5708, -1.5708, 1.0647, 0], time_from_start: {sec: 5, nanosec: 0.0}}]}, \
  goal_tolerance: [{name: \"shoulder_pan_joint\", position: 0.05}, \
  {name: \"shoulder_lift_joint\", position: 0.05}, {name: \"elbow_joint\", position: 0.05}, \
  {name: \"wrist_1_joint\", position: 0.05}, {name: \"wrist_2_joint\", position: 0.05}, \
  {name: \"wrist_3_joint\", position: 0.05}], \
  goal_time_tolerance: {sec: 1, nanosec: 0}}" ;

In [ ]:
# Move along trajectory to pickup location
! ros2 action send_goal /simulated_arm_server_1/follow_joint_trajectory \
  control_msgs/action/FollowJointTrajectory \
  "{trajectory: {joint_names: [\"shoulder_pan_joint\", \"shoulder_lift_joint\", \"elbow_joint\", \
  \"wrist_1_joint\", \"wrist_2_joint\", \"wrist_3_joint\"], \
  points: [{positions: [-1.0996, -2.0595, -0.76794, -1.8675, 1.6057, 0.29671], time_from_start: {sec: 5, nanosec: 0}}, \
           {positions: [-1.1519, -2.0944, -1.7802, -0.80285, 1.5882, 1.885], time_from_start: {sec: 10, nanosec: 0}}]}, \
  goal_tolerance: [{name: \"shoulder_pan_joint\", position: 0.05}, \
                   {name: \"shoulder_lift_joint\", position: 0.05}, \
                   {name: \"elbow_joint\", position: 0.05}, \
                   {name: \"wrist_1_joint\", position: 0.05}, \
                   {name: \"wrist_2_joint\", position: 0.05}, \
                   {name: \"wrist_3_joint\", position: 0.05}], \
  goal_time_tolerance: {sec: 1, nanosec: 0}}"

In [ ]:
# to move along a trajectory:
! ros2 action send_goal /simulated_arm_server_1/follow_joint_trajectory \
  control_msgs/action/FollowJointTrajectory \
  "{trajectory: {joint_names: [\"shoulder_pan_joint\", \"shoulder_lift_joint\", \"elbow_joint\", \
  \"wrist_1_joint\", \"wrist_2_joint\", \"wrist_3_joint\"], \
  points: [ \
    {positions: [0.0, 0.1, 0.2, -0.1, -0.2, 0.0], time_from_start: {sec: 4, nanosec: 0}}, \
    {positions: [0.1, 0.0, 0.1, 0.1, 0.0, 0.1], time_from_start: {sec: 8, nanosec: 0}}, \
    {positions: [-0.1, 0.1, 0.0, -0.1, 0.2, -0.2], time_from_start: {sec: 12, nanosec: 0}}]}, \
  goal_tolerance: [ \
    {name: \"shoulder_pan_joint\", position: 0.05}, \
    {name: \"shoulder_lift_joint\", position: 0.05}, \
    {name: \"elbow_joint\", position: 0.05}, \
    {name: \"wrist_1_joint\", position: 0.05}, \
    {name: \"wrist_2_joint\", position: 0.05}, \
    {name: \"wrist_3_joint\", position: 0.05}], \
  goal_time_tolerance: {sec: 1, nanosec: 0}}"

### Publishing a gripper control message
The gripper is controlled using a simple publisher (this means no feedback!). The command below shows how you can send a new control message. The values in the message are 32 bit ints (0-255) gripper distance, speed, and force. For the vizualization force doesnt do anything. 

In [ ]:
! ros2 topic pub -1 /simulated_arm_serverN/gripper/control \
std_msgs/msg/Int32MultiArray \
"{data: [255, 127, 127]}"

### Sending action goals with Python
Now try writing a python function using rclpy that sends this action goal to the simulated arm server node using the action client follow_joint_trajectory. Try combining this with the publishing code from the previous page to also control the gripper!

In [ ]:
import rclpy  # Import the ROS 2 Python client library
from rclpy.action import ActionClient  # For interacting with action servers
from rclpy.node import Node  # Base class for creating nodes
from control_msgs.action import FollowJointTrajectory  # Action type used in the goal
from trajectory_msgs.msg import JointTrajectoryPoint  # Message type for trajectory points

class TrajectoryActionClient(Node):
    def __init__(self):
        # Initialize the node with a name
        super().__init__('Node_Name')
        
        # Create an ActionClient for sending goals to the action server
        self._action_client = ActionClient(
            self, 
            FollowJointTrajectory, 
            '/simulated_arm_server_1/follow_joint_trajectory'
        )

    def send_goal(self):
        # Create a goal message for the FollowJointTrajectory action
        goal_msg = FollowJointTrajectory.Goal()
    
        # Specify the joint names for the robot's arm (these are standard for FollowJointTrajectory)
        goal_msg.trajectory.joint_names = [
            'shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint',
            'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint'
        ]
        
        # Create a JointTrajectoryPoint and set positions for each joint
        point = JointTrajectoryPoint()
        # IMPORTANT: Follow Joint Trajectory is set up to enable an input trajectory of joint angles but the simulation has not been tested. 
        # This means you should stick with sending one set of angles at a time although you can call the function several times to achieve trajectory control.
        # or you can try it and let me know how it works...
        point.positions = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0] # what if points were an input to the function?
        
        goal_msg.trajectory.points = [point]
        # Wait until the action server is available before sending the goal
        self._action_client.wait_for_server()

        # Send the goal and register a future callback to handle the response
        self._send_goal_future = self._action_client.send_goal_async(goal_msg)
        self._send_goal_future.add_done_callback(self.goal_response_callback)

    def goal_response_callback(self, future):
        # Handle the response from the action server
        goal_handle = future.result()
        
        # Check if the goal was accepted or rejected
        if not goal_handle.accepted:
            self.get_logger().info('Goal rejected :(')
            return
        
        self.get_logger().info('Goal accepted :)')

        # Register a callback to check when the result is available
        self._get_result_future = goal_handle.get_result_async()
        self._get_result_future.add_done_callback(self.get_result_callback)

    def get_result_callback(self, future):
        # Handle the result of the action
        result = future.result().result
        self.get_logger().info(f'Action completed with result: {result}')
        rclpy.shutdown()  # Shutdown the node once the result is processed

def main(args=None):
    # Initialize the rclpy library
    rclpy.init(args=args)

    # Create the action client node
    action_client = TrajectoryActionClient()

    # Send the goal
    action_client.send_goal()

    # Spin the node so it can process events (like receiving action results)
    rclpy.spin(action_client)

if __name__ == '__main__':
    main()


### Sending an action goal to the real arm
The command below can be used to control the real arm! It might look familiar... thats because it uses the same Follow Joint Trajectory action with the /scaled_joint_trajectory_controller/ namespace which indicates it is communicating with the UR arm driver. This means you could test out angles or a trajectory in the simulation and test it on the real arm by changing just a few lines of code, highlighting the modularity of ROS! Note: a bunch of other input parameters are included in this command as a (json dictionary) what are they? What do they do? Why might they be necessary for a real arm but not the simulated arm?

In [ ]:
! ros2 action send_goal /scaled_joint_trajectory_controller/follow_joint_trajectory \
  control_msgs/action/FollowJointTrajectory \
  "{trajectory: {joint_names: [\"shoulder_pan_joint\", \"shoulder_lift_joint\", \"elbow_joint\", \
  \"wrist_1_joint\", \"wrist_2_joint\", \"wrist_3_joint\"], \
  points: [ \
    {positions: [-1.1519, -1.7453, -0.57596, -2.2689, 1.5882, 1.885], time_from_start: {sec: 5, nanosec: 0}}, \
    {positions: [-0.89012, -1.8326, 1.1868, 0.66323, 2.1991, 1.9199], time_from_start: {sec: 15, nanosec: 0}}, \
    {positions: [-1.6406, -1.0821, 0.97738, 0.17453, 1.4137, 3.194], time_from_start: {sec: 20, nanosec: 0}}]}, \
  goal_tolerance: [ \
    {name: \"shoulder_pan_joint\", position: 0.05}, \
    {name: \"shoulder_lift_joint\", position: 0.05}, \
    {name: \"elbow_joint\", position: 0.05}, \
    {name: \"wrist_1_joint\", position: 0.05}, \
    {name: \"wrist_2_joint\", position: 0.05}, \
    {name: \"wrist_3_joint\", position: 0.05}], \
  goal_time_tolerance: {sec: 1, nanosec: 0}}"

### Controlling the real gripper
Just like with the real arm, the real gripper is controlled very similarly to the visualization just with a small change to the namespace -- namely, that it uses the global namespace /gripper/control.

In [ ]:
! ros2 topic pub -1 gripper/control \
std_msgs/msg/Int32MultiArray \
"{data: [255, 127, 127]}"